The csv file contains a set of data from one of our 3 phase energy monitoring sensors.   
Phase L1, L2 and L3 with the parameters real power, imaginary power, current and voltage.

[  ] Create a python dash application to display the data contained in the excel file  
The user visualizing the data shall be able to select which data he wants to display in the app

In [317]:
import pandas as pd
import plotly.express as px
import datetime as dt

from plotly.subplots import make_subplots
import plotly.graph_objects as go


import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [241]:
def prep_datetime(df):
    """
    convert day and time columns to a single date time column
    drop the original day and time columns
    """
    df['time'] = df['time'].apply(pd.Timedelta)
    df['datetime'] = pd.to_datetime(df['day']) + df['time']
    df = df.set_index('datetime')
    df = df.drop(columns=['day','time'])
    return df

In [330]:
def filter_df_cols(df, type_selection=['p','q','i','v']):
    
    selected_cols = []
    for col in df.columns:
        if col[-1] in type_selection:
            selected_cols.append(col)
    
    return df.loc[:,selected_cols]


In [407]:
ser = df['l2_i']
mu = ser.mean()
std = ser.std()

In [397]:
mu

125.89413333333333

In [398]:
std

26.469422033009753

In [405]:
ser.shape

(60,)

In [408]:
filter_outliers(ser).shape

-2.9213547492617695
3.4080547492617694


(59,)

In [403]:

ser.max()

191.285

In [366]:
def make_plot(df, type_selection):
    
    val_cols = df.columns.tolist()
    label_dict = {"p":"real power",
                  "q":"imaginary power",
                  "i":"current",x
                  "v":"voltage"}
    color_dict = {1:'red', 2:'blue', 3:'green'}
    subplot_titles = [f"{label_dict[sym]} ({sym})" for sym in type_selection]

    fig = make_subplots(rows=len(val_cols), cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.02,
                        subplot_titles=subplot_titles)

    legend_lst = [] #used to turn on and off add legend booleaN 

    for i, col in enumerate(val_cols):

        y=df[col]
        phase_num = int(col[1])
        val_type = col[-1]
        #is_non_zero used to start plot with zero series disabled 
        is_non_zero = (y.min() and y.max()) != 0
        visible_setting = True if is_non_zero else 'legendonly'
        # to ensure subplots are ordered as they are selected
        row_num = type_selection.index(val_type) +1
        color = color_dict[phase_num]
        legendgroup_name=f"phase {phase_num}"
        
        # to prevent legend names being added more than once
        if legendgroup_name not in legend_lst:
            legend_lst.append(legendgroup_name)
            showlegend = True
        else:
            showlegend = False

        props_dict={'color':color_dict[phase_num]}
        fig.add_trace(go.Scatter(x=y.index, y=y,
                                 legendgroup=legendgroup_name,
                                 name=legendgroup_name,
                                 showlegend=showlegend,
                                 visible=visible_setting,
                                 line=props_dict),
                      row=row_num, 
                      col=1)

    fig.update_layout(height=(200*(i+1)), width=600)
    
    return fig

In [417]:
import requests
url = "https://raw.githubusercontent.com/BarrySunderland/clemap-dash-demo/main/data/raw/output.csv?token=AK4OLGDAIK2ESYKGEEA4KN3AIVN4S"
resp = requests.get(url)

In [420]:
import os
os.listdir()

['data',
 'src',
 '.gitignore',
 'LICENSE',
 'README.md',
 '.git',
 'heroku.yml',
 '.ipynb_checkpoints',
 'notebooks',
 'requirements.txt',
 'Procfile']

In [324]:
#load data
fpath = './data/raw/output.csv'
df = pd.read_csv(fpath)
df = prep_datetime(df)

In [248]:
#change to long format for easier plotting
df = df.melt(var_name='measurement', id_vars='datetime')

df[['phase','measurement']] = df['measurement'].str.split('_', expand=True)

df['phase'] = df['phase'].str.replace('l','')



df['label'] = df['measurement'].replace(label_dict,regex=True)
df

In [367]:
# input_selection
input_selection = ['p','i','v']

In [368]:
tdf = filter_df_cols(df, type_selection=input_selection)
fig = make_plot(tdf, type_selection=input_selection)

In [369]:

fig.write_html('first_figure.html', auto_open=True)

In [147]:

unique_measurements=df['measurement'].unique()

for i, m in enumerate(unique_measurements):
    tdf = df.loc[df['measurement']==m,:].copy()
    tdf = tdf.rename(columns={'value':label_dict[m]})
    fig = px.line(tdf, 
                  x='datetime', 
                  y=label_dict[m], 
                  color='phase',)


In [256]:

fig = make_subplots(rows=len(val_cols), 
                    cols=1,
                    shared_xaxes=True,
                    subplot_titles=list(label_dict.values()))

In [263]:



for col in val_cols:
    
    phase_num = col[1]
    val_type = col[-1]
    
    print(phase_num)
    row_num = row_order[val_type]
    legend_group=f"phase {phase_num}"
    show_legend = True if phase_num == 1 else False
    y = df[col]
    visible = (y.max() and y.min()) != 0
    visible_setting = None if visible else 'legendonly'
    print(visible_setting)
    fig.add_trace(go.Scatter(x=df['datetime'],
                           y=y,
                           legendgroup=legend_group,
                           showlegend=show_legend,
                           name=col,
                           mode='lines',
                           visible=visible_setting),
                  row=row_num,
                  col=1)
    fig.update_layout(showlegend=True)
    

1
None
2
None
3
legendonly
1
None
2
None
3
legendonly
1
None
2
None
3
legendonly
1
None
2
None
3
legendonly


In [282]:

fig.write_html('first_figure.html', auto_open=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

/home/barry/anaconda3/envs/clemap-demo/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3445: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

